In [326]:
## Program to look at regions and region coordinates

In [1]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

exp = 'FR1'

poss_subs = df[df.experiment == exp]

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [2]:
# sub lists I've been using to explore FR1 

# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2
subs = ['R1112M','R1163T'] # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1368T','R1461T','R1463E'] # subs with particularly low localization.pair matches in pairs...see Excel in loading info on Box for details
# subs = ['R1433E','R1355T','R1367D','R1368T'] # subs with "ca3" or "dg"
# subs = ['R1065J'] # subject with over 8000 FR1 trials X electrodes
# bad_subs = ['R1433E','R1051J'] # no electrode cats

# sub lists to explore catFR

sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[1:2]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
639,NaN,protocols/r1/subjects/R1112M/experiments/FR1/s...,protocols/r1/subjects/R1112M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1112M/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1112M/localizations/0/m...,NaN,0,R1112M,R1112M,1.0,protocols/r1/subjects/R1112M/experiments/FR1/s...
640,NaN,protocols/r1/subjects/R1112M/experiments/FR1/s...,protocols/r1/subjects/R1112M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1112M/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1112M/localizations/0/m...,NaN,1,R1112M,R1112M,1.0,protocols/r1/subjects/R1112M/experiments/FR1/s...
641,NaN,protocols/r1/subjects/R1112M/experiments/FR1/s...,protocols/r1/subjects/R1112M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1112M/experiments/FR1/s...,0,NaN,2,protocols/r1/subjects/R1112M/localizations/0/m...,NaN,2,R1112M,R1112M,1.0,protocols/r1/subjects/R1112M/experiments/FR1/s...
924,NaN,protocols/r1/subjects/R1163T/experiments/FR1/s...,protocols/r1/subjects/R1163T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1163T/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1163T/localizations/0/m...,NaN,0,R1163T,R1163T,NaN,protocols/r1/subjects/R1163T/experiments/FR1/s...
925,NaN,protocols/r1/subjects/R1163T/experiments/FR1/s...,protocols/r1/subjects/R1163T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1163T/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1163T/localizations/0/m...,NaN,1,R1163T,R1163T,NaN,protocols/r1/subjects/R1163T/experiments/FR1/s...


In [3]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
if exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
exp_df.shape
exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(565, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
1,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,1,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
9,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,0,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
10,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,1,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
17,NaN,protocols/r1/subjects/R1003P/experiments/FR1/s...,protocols/r1/subjects/R1003P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1003P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1003P/localizations/0/m...,NaN,0,R1003P,R1003P,1.0,protocols/r1/subjects/R1003P/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,NaN,protocols/r1/subjects/R1565T/experiments/FR1/s...,protocols/r1/subjects/R1565T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1565T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1565T/localizations/0/m...,NaN,0,R1565T,R1565T,1.0,protocols/r1/subjects/R1565T/experiments/FR1/s...
2538,NaN,protocols/r1/subjects/R1569T/experiments/FR1/s...,protocols/r1/subjects/R1569T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1569T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1569T/localizations/0/m...,NaN,0,R1569T,R1569T,1.0,protocols/r1/subjects/R1569T/experiments/FR1/s...
2543,NaN,protocols/r1/subjects/R1571T/experiments/FR1/s...,protocols/r1/subjects/R1571T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1571T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1571T/localizations/0/m...,NaN,0,R1571T,R1571T,1.0,protocols/r1/subjects/R1571T/experiments/FR1/s...
2547,NaN,protocols/r1/subjects/R1572T/experiments/FR1/s...,protocols/r1/subjects/R1572T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1572T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1572T/localizations/0/m...,NaN,0,R1572T,R1572T,1.0,protocols/r1/subjects/R1572T/experiments/FR1/s...


## Check pairs and localizations for certain regions for given electrode

In [113]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload

### PARAMS ###

selected_period = 'surrounding_recall' # surrounding_recall # whole_retrieval # encoding 
# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0  # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY
# 3: ISOLATED only!

selected_region = ENT_labels #HPC_labels # ENT_labels+HPC_labels 

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal
min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.0 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 #???

filter_type = 'hamming' # 'butter' # Vaz used Butterworth from 80-120 Hz; Norman used Hamming from 70-180 Hz
# •Butterworth from Vaz et al: 2nd order from 80-120 ms, hilbert, select events >2 STD above mean of filtered traces.
#      Events >=25 ms long w/ max amp >3 SD were retained as ripples. Join adjacent ripples separated by <15 ms 
# •Hilbert from Norman et al: "70-180 Hz zero-lag linear-phase Hamming FIR filter w/ transition bandwidth of 5 Hz"
#      Then Hilbert, clip extreme to 4 SD, square this clipped, smooth w/ Kaiser FIR low-pass filter with 40 Hz cutoff,
#      mean and SD computed across entire experimental duration to define the threshold for event detection
#      Events from original (squared but unclipped) signal >4 SD above baseline were selected as candidate SWR events. 
#      Duration expanded until ripple power <2 SD. Events <20 ms or >200 ms excluded. Adjacent events <30 ms separation (peak-to-peak) merged.
# •Using IED detection from Vaz for 250 Hz highpass events (really 250-499). Norman uses 20-60 Hz events which is...odd.
#       See detectRipples code in module for this algorithm


# recall params
recall_minimum = 2000
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**

### END PARAMS ###

if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time

ripple_array = []; fr_array = []; 
trial_nums = []; 
session_ct = 0; channel_ct = 0; total_channel_ct = 0
HPC_names = []; sub_names = []; sub_sess_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = [];             
encoded_word_key_array = []; serialpos_array = [] # encoding info
ordinal_position_array = []

trial_by_trial_correlation = []; elec_by_elec_correlation = []
elec_ripple_rate_array = []

all_regions = []
all_atlas = []
all_pair_numbers = []
all_stein_das = []
non_locs = 0

# to get atlas coordinate info
ct=0
coord_atlases = ['avg.x','avg.corrected.x','ind.x','ind.corrected.x','mni.x']
coord_matrix = np.empty((len(exp_df),len(coord_atlases))); coord_matrix[:] = np.nan
region_matrix = np.matrix(coord_matrix,dtype='object')

for row in exp_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (from this list...no instrusions). 
        
        # need to remove free recalls that happened more than once        
        word_evs = evs[evs['type']=='WORD'] # get words 
        evs_free_recall,_ = removeRepeatedRecalls(evs_free_recall,word_evs)
        # select which recalls??         
        [recall_selection_name,selected_recalls_idxs] = selectRecallType(recall_type_switch,evs_free_recall,IRI,recall_minimum)
        evs_free_recall = evs_free_recall[selected_recalls_idxs]
        
        # let's make sure remaining recalls have eeg 
        evs_free_recall = evs_free_recall[evs_free_recall.eegoffset > -1]
        if len(evs_free_recall)==0: #...and that any are left!
            break
            
        # get output position in each list for this session's recalls
        session_corrected_list_ops = getOutputPositions(evs,evs_free_recall)
            
        # get pairs and loc pairs
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
        except:
            localizations = []
            non_locs+=1
        
        channel = 10
        
        # get what coord_atlases have coordinates for this channel
        for i,atlas_name in enumerate(coord_atlases):
            if atlas_name in pairs:
                coord_matrix[ct,i] = pairs.iloc[channel][atlas_name] # coord_matrix will have coordinates for the 5 coord_atlases listed above
        
        # same but check for regions in atlas
        for i,atlas_name in enumerate(['dk.region','wb.region']):
            if atlas_name in pairs.iloc[channel]:
                region_matrix[ct,i] = pairs.iloc[channel][atlas_name]        
        if len(localizations) > 1:
            for i,atlas_name in enumerate(['atlases.dk','atlases.whole_brain']):
                if atlas_name in localizations:
                    region_matrix[ct,i+2] = localizations[atlas_name]['pairs'][channel]
        ct+=1
                
        # get translation from loc to pairs
        if len(localizations) > 1:
            loc_translation = Loc2PairsTranslation(pairs,localizations)
            loc_pairs = localizations.type.pairs
            num_translations = sum([loc!=' ' for loc in loc_translation])
            if len(pairs) >= len(loc_pairs):
                if num_translations != len(loc_pairs):
                    print(sub+'-'+str(session)+'-more pairs; len(pairs): '+str(len(pairs))+', len(loc_pairs): '+str(len(loc_pairs))+', trans: '+str(num_translations))
            elif len(pairs) < len(loc_pairs):
                if num_translations != len(pairs):
                    print(sub+'-'+str(session)+'-more loc_pairs; len(loc_pairs): '+str(len(loc_pairs))+', len(pairs): '+str(len(pairs))+', trans: '+str(num_translations))
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 
        
        # to view all loc_pairs at once: [loc_pairs[lp:lp+1] for lp in range(len(loc_pairs)-1)]
        # to view all pairs at once: [pairs[p:p+1].label for p in range(len(pairs)-1)]
        
        all_regions.extend(elec_regions)
        all_atlas.extend(atlas_type)
        all_pair_numbers.extend(pair_number)
        all_stein_das.append(has_stein_das)
        sub_sess_names.extend(np.tile(sub+'-'+str(session),len(elec_regions))) 

    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue



/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data

2021-01-22_14-49-48: DF Exception: Sub: R1093J, Sess: 0, TypeError, cannot convert the series to <class 'int'>, file: <ipython-input-113-303502364257>, line no: 102


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1094T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1094T/tal/R1094T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could not load MNI coordinates
  "Could not load MNI coordinates"))
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1094T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1094T/tal/R1094T_ta

KeyboardInterrupt: 

In [136]:
# to view which rows have data in region_matrix
# atlas order in columns: 0) pairs dk 1) pairs wb 2) loc dk 3) loc wb
column = 3 # column to check atlas order in region_mask
region_mask = [pd.isnull(row[0,0]) for row in region_matrix[:,column]] # unpack matrix
region_mask = np.invert(region_mask)
len(exp_df.subject[region_mask])
exp_df.subject[region_mask][0:20]
exp_df.subject[region_mask][-20:]
# region_matrix[:,3]

123

1122    R1198M
1125    R1200T
1754    R1341T
1755    R1341T
1757    R1342M
1761    R1345D
1762    R1345D
1763    R1345D
1764    R1345D
1765    R1345D
1770    R1346T
1771    R1346T
1772    R1346T
1773    R1347D
1774    R1347D
1775    R1347D
1782    R1349T
1783    R1350D
1784    R1350D
1785    R1350D
Name: subject, dtype: object

2048    R1423E
2050    R1425D
2051    R1425D
2052    R1425D
2058    R1427T
2059    R1427T
2060    R1427T
2063    R1431J
2064    R1433E
2066    R1436J
2070    R1438M
2071    R1441T
2072    R1441T
2073    R1441T
2077    R1443D
2078    R1443D
2094    R1446T
2095    R1447M
2097    R1448T
2100    R1449T
Name: subject, dtype: object

## Of 547 FR1 sessions from R1001P to R1467M ## 

## FR1 channel 10 ##
pairs.json:
'dk': R1001P to R1338T, 134 sessions have region name
'wb': R1030J to R1346T, 179 sessions have region name
localization.json:
'dk': R1204T to R1461T, 133 sessions have region name
'whole_brain': R1204T to R1461T, 122 sessions have region name

## FR1 channel 40 ##
pairs.json:
'dk': R1001P to R1336T, 142 sessions have region name
'wb': R1030J to R1342M, 200 sessions have region name
localization.json:
'dk': R1198M to R1449T, 133 sessions have region name
'whole_brain': R1198M to R1449T, 123 sessions have region name

## FR1 channel 70 ##
pairs.json:
'dk': R1003P to R1311T, 125 sessions have region name
'wb': R1026D to R1318N, 179 sessions have region name
localization.json:
'dk': R1173J to R1402E, 129 sessions have region name
'whole_brain': R1173J to R1402E, 122 sessions have region name


In [147]:
len(sub_sess_names)
# sub_sess_names[:20010]
np.unique(all_atlas[:20000],return_counts=True)

65257

(array(['No atlas', 'das', 'dk', 'ind', 'stein', 'wb'], dtype='<U8'),
 array([ 480,   36, 7401, 6267, 1459, 4357]))

In [152]:
sum(~np.isnan(coord_matrix[:,0])) # avg
len(coord_matrix)
# Atlas coordinates for FR1 (looking at Channel 10…channel 30 e.g. is different)
# avg: all 547 have
# avg.corrected: 132/547 subjects have (after ~R1341)
# ind: 546/547 have
# ind.corrected: 132/547 have (after ~R1341)
# mni: 410/553 have (missing ~half of first half of subjects)
# coord_matrix[:,4]

480

481

In [10]:
len(all_atlas)
len(all_regions)
# np.unique(all_atlas)
# np.unique(all_regions)
len(all_stein_das)

# what sessions have stein and das annotated?
np.array(all_stein_das)

# # how often is "MTL" atlas used?
# np.array(all_regions)[np.array(all_atlas)=='MTL_localization']

# what % of sessions have localization.json?
(len(exp_df)-non_locs)/len(exp_df)*100

67702

67702

488

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

59.100204498977504

In [16]:
# display coord_atlases with given region:
region = '"erc"'
len(np.array(all_atlas)[np.array(all_regions)==region])
np.array(all_atlas)[np.array(all_regions)==region]

print('left and right "hippocampus" '+str(160+93+43+118)+' electrodes') # using 'left hippocampus', ' left hippocampus' and same for right
print('left and right "amygdala" '+str(152+96+7+8)+' electrodes') # same with amygdala
(160+93+43+118)/len(all_atlas)*100
(152+96+7+8)/len(all_atlas)*100
print('For comparison: ')
print('left and right ca1 '+str(7+150+547+379)+' electrodes')
print('left and right entorhinal '+str(3+103+59+17+80+21)+' electrodes') # '"erc"', 'entorhinal' and then left/right names from below
(7+150+547+379)/len(all_atlas)*100
(3+103+59+17+80+21)/len(all_atlas)*100

10

array(['MTL_localization', 'MTL_localization', 'MTL_localization',
       'MTL_localization', 'MTL_localization', 'MTL_localization',
       'MTL_localization', 'MTL_localization', 'MTL_localization',
       'MTL_localization'], dtype='<U16')

left and right "hippocampus" 414 electrodes
left and right "amygdala" 263 electrodes


0.6115033529290125

0.38846710584620836

For comparison: 
left and right ca1 1083 electrodes
left and right entorhinal 283 electrodes


1.5996573217925616

0.4180083306253878

In [224]:
# subs with particular region 
np.unique(np.array(sub_sess_names)[np.array(all_regions)==region])

array(['R1207J-0', 'R1207J-1', 'R1347D-2', 'R1347D-3', 'R1347D-4',
       'R1354E-0', 'R1354E-1', 'R1355T-0', 'R1355T-1', 'R1355T-2',
       'R1361C-0', 'R1361C-1', 'R1367D-0', 'R1367D-1', 'R1373T-0',
       'R1376D-0', 'R1379E-0', 'R1379E-1', 'R1383J-0', 'R1385E-1',
       'R1385E-2', 'R1385E-3', 'R1387E-0', 'R1387E-1', 'R1387E-2',
       'R1394E-0', 'R1394E-1', 'R1395M-0', 'R1395M-1', 'R1395M-2',
       'R1395M-3', 'R1395M-4', 'R1395M-5', 'R1398J-1', 'R1398J-3',
       'R1402E-0', 'R1405E-0', 'R1405E-1', 'R1414E-0', 'R1415T-0',
       'R1415T-1', 'R1423E-0', 'R1425D-0', 'R1425D-1', 'R1425D-2',
       'R1436J-1', 'R1447M-1', 'R1449T-0', 'R1449T-1', 'R1454M-0',
       'R1463E-0', 'R1463E-1', 'R1467M-0'], dtype='<U10')

## Get regions from every electrode

In [5]:
%autoreload

# handles strings, float8, and float64
def isNaN(num):
    return num != num

### PARAMS ###

all_subs = np.unique(df.subject)
# all_subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T','R1461T','R1463E']


# to get atlas coordinate info

# coord_atlases = ['avg.x','avg.corrected.x','ind.x','ind.corrected.x','mni.x']
# pairs_regions = np.empty((len(exp_df),len(coord_atlases))); pairs_regions[:] = np.nan
# region_matrix = np.matrix(coord_matrix,dtype='object')

pairs_atlases = ['stein.region','das.region','wb.region','dk.region','ind.corrected.region',
                 'ind.region','avg.region','avg.corrected.region','hcp.region','mni.region','vox.region']
locs_atlases = ['atlases.mtl','atlases.dk','atlases.whole_brain','atlases.dkavg', 'atlases.manual']

pairs_regions = np.empty((len(all_subs),len(pairs_atlases))); pairs_regions[:] = np.nan
locs_regions = np.empty((len(all_subs),len(locs_atlases))); locs_regions[:] = np.nan

ct=0

for sub in all_subs:
    sub_df = df[df.subject==sub]    
    sub_df = sub_df[0:1] # take first session for every subject...not that this is assuming the same event creation was run on each session
    # for this subject, as well as for each montage/localization for this subject
    
    for row in sub_df.itertuples():  
#         try:
            sub = row.subject; session = row.session; exp = row.experiment
            mont = int(row.montage); loc = int(row.localization)
            reader = CMLReadDFRow(row)
            
            # get pairs and loc pairs
            pairs = reader.load('pairs')
            try:
                localizations = reader.load('localization')
            except:
                localizations = []

                
            # what regions in pairs atlases?
            
            pairs_channel_range = np.arange(0,len(pairs))
#             pairs_channel_range = np.arange(0,10*np.floor(len(pairs)/10),10) # every 10 channels 
            
            for i,atlas_name in enumerate(pairs_atlases):
                
                has_region = 0
                for channel in pairs_channel_range:
                    if atlas_name in pairs.iloc[int(channel)]:
                        region_name = pairs.iloc[int(channel)][atlas_name]
                        if isNaN(region_name) == False: # will crash below if nan is a float
                            if (region_name is not None) and (len(region_name)>1) and \
                               (region_name not in 'None') and (region_name != 'nan'):
                                has_region = 1                            
                    pairs_regions[ct,i] = has_region                  
            
            if len(localizations) > 1:
    
                locs_channel_range = np.arange(len(localizations['atlases.whole_brain']['pairs']))
#                 locs_channel_range = np.arange(0,10*np.floor(len(localizations['atlases.whole_brain']['pairs'])/10),10) # every 10 channels
                                            
                for i,atlas_name in enumerate(locs_atlases):
                    
                    has_region = 0
                    for channel in locs_channel_range:
                        if atlas_name in localizations:
                            region_name = localizations[atlas_name]['pairs'][int(channel)]
                            if isNaN(region_name) == False: # will crash below if nan is a float
                                if (region_name is not None) and (len(region_name)>1) and \
                                   (region_name not in 'None') and (region_name != 'nan'):
                                    has_region = 1
                        locs_regions[ct,i] = has_region
                        
#             # get what coord_atlases have coordinates for this channel
#             for i,atlas_name in enumerate(coord_atlases):
#                 if atlas_name in pairs:
#                     coord_matrix[ct,i] = pairs.iloc[channel][atlas_name] # coord_matrix will have coordinates for the 5 coord_atlases listed above

            ct+=1

#             # get translation from loc to pairs to test if Loc2PairsTranslation works
#             if len(localizations) > 1:
#                 loc_translation = Loc2PairsTranslation(pairs,localizations)
#                 loc_pairs = localizations.type.pairs
#                 num_translations = sum([loc!=' ' for loc in loc_translation])
#                 if len(pairs) >= len(loc_pairs):
#                     if num_translations != len(loc_pairs):
#                         print(sub+'-'+str(session)+'-more pairs; len(pairs): '+str(len(pairs))+', len(loc_pairs): '+str(len(loc_pairs))+', trans: '+str(num_translations))
#                 elif len(pairs) < len(loc_pairs):
#                     if num_translations != len(pairs):
#                         print(sub+'-'+str(session)+'-more loc_pairs; len(loc_pairs): '+str(len(loc_pairs))+', len(pairs): '+str(len(pairs))+', trans: '+str(num_translations))

#             tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
#             elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 

#             # to view all loc_pairs at once: [loc_pairs[lp:lp+1] for lp in range(len(loc_pairs)-1)]
#             # to view all pairs at once: [pairs[p:p+1].label for p in range(len(pairs)-1)]

#             all_regions.extend(elec_regions)
#             all_atlas.extend(atlas_type)
#             all_pair_numbers.extend(pair_number)
#             all_stein_das.append(has_stein_das)
#             sub_sess_names.extend(np.tile(sub+'-'+str(session),len(elec_regions))) 

#         except Exception as e:
#             LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
#             continue



/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1002P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1002P/tal/R1002P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1003P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1003P/tal/R1003P_talLocs_database_bipol.mat returning the 

KeyboardInterrupt: 

In [81]:
np.savetxt("../compiled/localization/all_RAM_subjects_names.csv",np.asarray(all_subs), delimiter=",",fmt='%s')
np.savetxt("../compiled/localization/all_RAM_subjects_pairs_regions.csv",np.asarray(pairs_regions), delimiter=",")
np.savetxt("../compiled/localization/all_RAM_subjects_locs_regions.csv",np.asarray(locs_regions), delimiter=",")
np.savetxt("../compiled/localization/pair_atlas_names.csv",np.asarray(pairs_atlases), delimiter=",",fmt='%s')
np.savetxt("../compiled/localization/localization_atlas_names.csv",np.asarray(locs_atlases), delimiter=",",fmt='%s')

## I want to get every region name for every electrode in a single array (and the atlas too)

In [6]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload

all_regions = []
all_atlas = []
all_pair_numbers = []
all_stein_das = []
non_locs = 0
sub_sess_names = []

# to get atlas coordinate info
ct=0
coord_atlases = ['avg.x','avg.corrected.x','ind.x','ind.corrected.x','mni.x']
coord_matrix = np.empty((len(exp_df),len(coord_atlases))); coord_matrix[:] = np.nan
region_matrix = np.matrix(coord_matrix,dtype='object')

for row in df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
            
        # get pairs and loc pairs
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
        except:
            localizations = []
            non_locs+=1
        
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 

        # to view all loc_pairs at once: [loc_pairs[lp:lp+1] for lp in range(len(loc_pairs)-1)]
        # to view all pairs at once: [pairs[p:p+1].label for p in range(len(pairs)-1)]

        all_regions.extend(elec_regions)
        all_atlas.extend(atlas_type)
        all_pair_numbers.extend(pair_number)
        all_stein_das.append(has_stein_das)
        sub_sess_names.extend(np.tile(sub+'-'+str(session),len(elec_regions))) 

    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue



/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.4, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pair

2021-10-06_11-58-26: DF Exception: Sub: R1535E, Sess: 0, KeyError, "None of [Index(['contact_1', 'contact_2', 'label'], dtype='object')] are in the [columns]", file: <ipython-input-6-ee1117d01d3b>, line no: 26


In [23]:
all_regions = np.unique(all_regions)
all_atlas = np.unique(all_atlas)
sub_sess_names = np.unique(sub_sess_names)
np.savetxt("../compiled/localization/pair_region_names.csv",np.asarray(all_regions), delimiter=",",fmt='%s')
np.savetxt("../compiled/localization/pair_atlas_names.csv",np.asarray(all_atlas), delimiter=",",fmt='%s')
np.savetxt("../compiled/localization/sub_sess_names.csv",np.asarray(sub_sess_names), delimiter=",",fmt='%s')


In [249]:
# display coord_atlases with given region:

# (what I used to populate MTL atlas breakdown on RAM localization tracker)
region = 'left hippocampus' # lowercase here
print('Number of electrodes:')
len(np.array(all_atlas)[np.array(all_regions)==region])
# np.array(all_atlas)[np.array(all_regions)==region]
np.array(sub_sess_names)[np.array(all_regions)==region]

Number of electrodes:


93

array(['R1033D-0', 'R1035M-0', 'R1035M-0', 'R1035M-0', 'R1044J-0',
       'R1045E-0', 'R1054J-0', 'R1054J-1', 'R1086M-0', 'R1101T-0',
       'R1101T-0', 'R1102P-0', 'R1102P-0', 'R1120E-1', 'R1120E-1',
       'R1120E-1', 'R1120E-1', 'R1120E-1', 'R1122E-0', 'R1122E-0',
       'R1122E-1', 'R1122E-1', 'R1134T-0', 'R1134T-1', 'R1134T-2',
       'R1137E-0', 'R1137E-1', 'R1137E-2', 'R1172E-0', 'R1172E-1',
       'R1172E-2', 'R1174T-0', 'R1174T-1', 'R1187P-0', 'R1187P-1',
       'R1195E-0', 'R1195E-0', 'R1195E-0', 'R1195E-1', 'R1195E-1',
       'R1195E-1', 'R1195E-2', 'R1195E-2', 'R1195E-2', 'R1203T-0',
       'R1221P-1', 'R1221P-1', 'R1221P-1', 'R1221P-1', 'R1247P-0',
       'R1264P-0', 'R1268T-0', 'R1281E-1', 'R1281E-2', 'R1281E-3',
       'R1286J-0', 'R1291M-0', 'R1291M-0', 'R1293P-3', 'R1293P-3',
       'R1293P-4', 'R1293P-4', 'R1299T-0', 'R1299T-1', 'R1299T-4',
       'R1299T-5', 'R1299T-6', 'R1299T-7', 'R1299T-8', 'R1299T-9',
       'R1323T-0', 'R1323T-2', 'R1323T-3', 'R1325C-0', 'R1325C

In [266]:
# see specific atlas for one patient and compare a pairs to a localizations atlas
sub = ['R1033D']
session = 0
sub_df = df[(df.subject.isin(sub))  & (df.experiment == exp) & (df.session == session)]
sub_df
for row in sub_df.itertuples():  
    sub = row.subject; session = row.session; exp = row.experiment
    mont = int(row.montage); loc = int(row.localization)
    reader = CMLReadDFRow(row)

    # get pairs and loc pairs
    pairs = reader.load('pairs')
    try:
        localizations = reader.load('localization')
    except:
        localizations = []
        
        
# define pairs region to look for to compare
region2 = 'Left Hippocampus' # Left Ent entorhinal area # uppercases needed here
atlas2 = 'wb.region' #'stein.region'

atlas_regions2 = np.array(pairs[atlas2])
print('Length of atlas 2 pairs: '+str(len(atlas_regions2)))
print('Sites with labels:')
sites = [site == region2 for site in atlas_regions2]
atlas2_idxs = np.where(sites)[0]
atlas2_idxs
pairs.label[atlas2_idxs]
pairs.iloc[atlas2_idxs]
pairs.iloc[109:120]['stein.region']
# # define localizations region to look for
# region1 = 'left ent entorhinal area' # '"ERC"'
# atlas_regions1 = np.array(localizations['atlases.mtl'])
# atlas_regions1[0:30]
# print('Length of atlas 1 pairs: '+str(len(atlas_regions1)))
# print('Sites with labels:')
# sites = [site == region1 for site in atlas_regions1]
# atlas1_idxs = np.where(sites)[0]
# atlas1_idxs
# localizations.iloc[atlas1_idxs]

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
172,NaN,protocols/r1/subjects/R1033D/experiments/FR1/s...,protocols/r1/subjects/R1033D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1033D/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1033D/localizations/0/m...,NaN,0,R1033D,R1033D,1.0,protocols/r1/subjects/R1033D/experiments/FR1/s...


Length of atlas 2 pairs: 68
Sites with labels:


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1033D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1033D/tal/R1033D_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


array([58, 59, 60, 61, 62])

58    LOTD2-LOTD3
59    LOTD3-LOTD4
60    LOTD4-LOTD5
61    LOTD5-LOTD6
62    LOTD6-LOTD7
Name: label, dtype: object

,contact_1,contact_2,label,id,is_explicit,is_stim_only,type_1,type_2,avg.region,avg.x,avg.y,avg.z,avg.dural.region,avg.dural.x,avg.dural.y,...,stein.region,stein.x,stein.y,stein.z,wb.region,wb.x,wb.y,wb.z,das.region,das.x,das.y,das.z,mni.x,mni.y,mni.z
58,73,74,LOTD2-LOTD3,lotd.2-lotd.3,False,False,D,D,None,-25.790,-9.105,-24.370,None,-25.790,-9.105,...,Left CA1,NaN,NaN,NaN,Left Hippocampus,NaN,NaN,NaN,CA1,NaN,NaN,NaN,-25.22400,-4.015953,-27.108514
59,74,75,LOTD3-LOTD4,lotd.3-lotd.4,False,False,D,D,None,-27.550,-17.485,-18.720,None,-27.550,-17.485,...,Left DG,NaN,NaN,NaN,Left Hippocampus,NaN,NaN,NaN,DG,NaN,NaN,NaN,-27.69615,-12.949405,-20.636680
60,75,76,LOTD4-LOTD5,lotd.4-lotd.5,False,False,D,D,None,-29.355,-25.865,-13.425,None,-29.355,-25.865,...,Left CA3,NaN,NaN,NaN,Left Hippocampus,NaN,NaN,NaN,DG,NaN,NaN,NaN,-29.46585,-21.986550,-14.714936
61,76,77,LOTD5-LOTD6,lotd.5-lotd.6,False,False,D,D,None,-30.955,-33.765,-8.155,None,-30.955,-33.765,...,Left DG,NaN,NaN,NaN,Left Hippocampus,NaN,NaN,NaN,DG,NaN,NaN,NaN,-30.33645,-30.490750,-8.833772
62,77,78,LOTD6-LOTD7,lotd.6-lotd.7,False,False,D,D,lingual,-32.550,-41.660,-2.885,lingual,-32.550,-41.660,...,NaN,NaN,NaN,NaN,Left Hippocampus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-31.60010,-38.844800,-3.668460


Series([], Name: stein.region, dtype: object)

In [257]:
np.array(pairs['stein.region'])
np.array(pairs['wb.region'])
pairs.iloc[:30]

array([nan, 'Right Sub', 'Right Sub', 'Right CA1', 'Right CA1', nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'Left PRC',
       'Left CA1', 'Left DG', 'Left CA3', 'Left DG', nan, nan, nan, nan,
       nan, nan], dtype=object)

array(['Right Cerebral White Matter', 'Right Hippocampus',
       'Right Hippocampus', 'Right Hippocampus', 'Right Hippocampus',
       'Right Lateral Ventricle', 'Right Cerebral White Matter',
       'Right Cerebral White Matter', 'Right MOG middle occipital gyrus',
       'Right MOG middle occipital gyrus', nan,
       'Right AIns anterior insula', 'Right PIns posterior insula',
       'Right PIns posterior insula', 'Right PO parietal operculum',
       'Right Cerebral White Matter', 'Right Cerebral White Matter',
       'Right Cerebral White Matter', 'Right Cerebral White Matter',
       'Right Cerebral White Matter', 'Right Cerebral White Matter', nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan,
       'Left Ent entorhinal area', 'Left Hippocampus', 'Left Hippocampus',
       'Left Hippocampus', 'Left Hippocampus', 'Left Hippocampus

,contact_1,contact_2,label,id,is_explicit,is_stim_only,type_1,type_2,avg.region,avg.x,avg.y,avg.z,avg.dural.region,avg.dural.x,avg.dural.y,...,stein.region,stein.x,stein.y,stein.z,wb.region,wb.x,wb.y,wb.z,das.region,das.x,das.y,das.z,mni.x,mni.y,mni.z
0,1,2,ROTD1-ROTD2,rotd.1-rotd.2,False,False,D,D,entorhinal,20.465,-7.445,-27.495,entorhinal,20.465000,-7.445000,...,NaN,NaN,NaN,NaN,Right Cerebral White Matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.00575,-3.007885,-30.661390
1,2,3,ROTD2-ROTD3,rotd.2-rotd.3,False,False,D,D,None,22.240,-15.690,-22.695,None,22.240000,-15.690000,...,Right Sub,NaN,NaN,NaN,Right Hippocampus,NaN,NaN,NaN,SUB,NaN,NaN,NaN,23.06855,-11.498155,-25.226681
2,3,4,ROTD3-ROTD4,rotd.3-rotd.4,False,False,D,D,None,24.325,-23.935,-17.210,None,24.325000,-23.935000,...,Right Sub,NaN,NaN,NaN,Right Hippocampus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.58105,-20.595300,-18.805313
3,4,5,ROTD4-ROTD5,rotd.4-rotd.5,False,False,D,D,None,26.845,-31.695,-11.790,None,26.845000,-31.695000,...,Right CA1,NaN,NaN,NaN,Right Hippocampus,NaN,NaN,NaN,CA1,NaN,NaN,NaN,28.29465,-29.664650,-12.019489
4,5,6,ROTD5-ROTD6,rotd.5-rotd.6,False,False,D,D,None,29.140,-39.465,-6.340,None,29.140000,-39.465000,...,Right CA1,NaN,NaN,NaN,Right Hippocampus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.16840,-38.619800,-5.250336
5,6,7,ROTD6-ROTD7,rotd.6-rotd.7,False,False,D,D,lingual,31.650,-46.985,-0.915,lingual,31.650000,-46.985000,...,NaN,NaN,NaN,NaN,Right Lateral Ventricle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.36570,-46.664450,0.326482
6,7,8,ROTD7-ROTD8,rotd.7-rotd.8,False,False,D,D,lingual,34.185,-55.210,4.505,lingual,34.185000,-55.210000,...,NaN,NaN,NaN,NaN,Right Cerebral White Matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.04395,-55.152150,7.086373
7,8,9,ROTD8-ROTD9,rotd.8-rotd.9,False,False,D,D,inferiorparietal,36.675,-63.435,9.570,inferiorparietal,36.675000,-63.435000,...,NaN,NaN,NaN,NaN,Right Cerebral White Matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.34335,-63.203400,13.499855
8,9,10,ROTD9-ROTD10,rotd.9-rotd.10,False,False,D,D,inferiorparietal,39.075,-71.655,13.920,inferiorparietal,39.075000,-71.655000,...,NaN,NaN,NaN,NaN,Right MOG middle occipital gyrus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.29505,-71.348950,17.638858
9,10,11,ROTD10-ROTD11,rotd.10-rotd.11,False,False,D,D,inferiorparietal,41.925,-79.860,18.210,inferiorparietal,41.925000,-79.860000,...,NaN,NaN,NaN,NaN,Right MOG middle occipital gyrus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.41175,-79.944000,21.685715


In [54]:
len(all_atlas)
len(all_regions)
# np.unique(all_atlas)
# np.unique(all_regions)
len(all_stein_das)

# what sessions have stein and das annotated?
# np.array(all_stein_das)

# # how often is "MTL" atlas used?
# np.array(all_regions)[np.array(all_atlas)=='MTL_localization']

# what % of sessions have localization.json?
(len(exp_df)-non_locs)/len(exp_df)*100

66184

66184

554

24.187725631768952